In [1]:
import os
os.environ['TF_USE_LEGACY_KERAS'] = 'True'

# Now import tensorflow/keras modules
import deepchem as dc
import tensorflow as tf
import tensorflow.keras.layers as layers
import numpy as np

No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
No normalization for NumAmideBonds. Feature removed!
No normalization for NumAtomStereoCenters. Feature removed!
No normalization for NumBridgeheadAtoms. Feature removed!
No normalization for NumHeterocycles. Feature removed!
No normalization for NumSpiroAtoms. Feature removed!
No normalization for NumUnspecifiedAtomStereoCenters. Feature removed!
No normalization for Phi. Feature removed!
No normalization for AvgIpc. Feature removed!
No normalization for NumAmideBonds. Feature removed!
No normalization for NumAtomStereoCenters. Feature removed!
No normalization for NumBridgeheadAtoms. Feature removed!
No normalization for NumHeterocycles. Feature removed!
No normalization for NumSpiroAtoms. Feature removed!
No normalization for NumUnspecifiedAtomStereoCenters. Feature removed!
No normalization for Phi. Feature removed!
c:\Users\cjulg\AppData\Local\Programs\Python\Python310\lib\site-packages\google


Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead
Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (c:\Users\cjulg\AppData\Local\Programs\Python\Python310\lib\site-packages\deepchem\models\torch_models\__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (c:\Users\cjulg\AppData\Local\Programs\Python\Python310\lib\site-packages\deepchem\models\torch_models\__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'lightning'
Skipped loading some Jax models, missing a dependency. No module named

In [2]:
import os 
import re

image_dir = 'BBBC005_v1_images'
files = []
labels = []

for file in os.listdir(image_dir):
    if file.endswith('.TIF'):
        files.append(os.path.join(image_dir, file))
        labels.append(int(re.findall('_C(.*?)_', file)[0]))
dataset = dc.data.ImageDataset(files, np.array(labels))


In [3]:
splitter = dc.splits.RandomSplitter()
train_dataset, valid_dataset, test_dataset = splitter.train_valid_test_split(dataset, seed=123)

In [4]:
features = tf.keras.Input(shape = (520,696,1))
prev_layer = features
for num_outputs in [16,32,64,128,256]:
    prev_layer = layers.Conv2D(num_outputs, kernel_size=5, strides=2,
                               activation=tf.nn.relu)(prev_layer)
output = layers.Dense(1)(layers.Flatten()(prev_layer))

keras_model = tf.keras.Model(inputs=features, outputs=output)
learning_rate = dc.models.optimizers.ExponentialDecay(0.001, 0.9, 250)
model = dc.models.KerasModel(
    keras_model,
    loss = dc.models.losses.L2Loss(),
    learning_rate = learning_rate,
    model_dir='model'
)



In [5]:
y_pred = model.predict(test_dataset).flatten()
print(np.sqrt(np.mean((y_pred-test_dataset.y)**2)))


51.27269986965928


In [ ]:
model.fit(train_dataset, nb_epoch=50)